In [ ]:
def simulate_normal(n_samples, cov_matrix, fix_method=None):
    """
    使用协方差矩阵模拟多元正态分布样本。

    参数:
        n_samples (int): 样本数量。
        cov_matrix (numpy.ndarray): 协方差矩阵。
        fix_method (callable, optional): 修正矩阵的方法，默认不修正。

    返回:
        numpy.ndarray: 模拟的多元正态分布样本。
    """
    if fix_method is not None:
        cov_matrix = fix_method(cov_matrix)  # 修正矩阵为半正定

    mean = np.zeros(cov_matrix.shape[0])  # 均值为 0
    return np.random.multivariate_normal(mean, cov_matrix, size=n_samples)

In [ ]:
def simulate_pca(cov_matrix, n_samples, pct_exp=0.99):
    """
    使用 PCA 对协方差矩阵降维并模拟样本。

    参数:
        cov_matrix (numpy.ndarray): 协方差矩阵。
        n_samples (int): 样本数量。
        pct_exp (float): 保留的累计解释方差百分比。

    返回:
        numpy.ndarray: 模拟的多元正态分布样本。
    """
    eigvals, eigvecs = np.linalg.eigh(cov_matrix)  # 特征值和特征向量
    sorted_indices = np.argsort(eigvals)[::-1]  # 从大到小排序
    eigvals = eigvals[sorted_indices]
    eigvecs = eigvecs[:, sorted_indices]

    # 累计解释方差
    explained_variance_ratio = np.cumsum(eigvals) / np.sum(eigvals)
    num_components = np.searchsorted(explained_variance_ratio, pct_exp) + 1

    # 降维
    reduced_eigvals = eigvals[:num_components]
    reduced_eigvecs = eigvecs[:, :num_components]
    reduced_cov_matrix = reduced_eigvecs @ np.diag(reduced_eigvals) @ reduced_eigvecs.T

    return simulate_normal(n_samples, reduced_cov_matrix)